In [13]:
import sys
import json
sys.path.append('..')
from src.analysis.model_performances import *
import pandas as pd
EVERYTHING_POSTFIX = 'everything'

In [23]:
def to_df(results_all, model_name, answer_type, prompt_type, subs, domains):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER_TYPE : 'T/F',
        FREE_ANSWER_TYPE : 'Free'
    }
    
    index = []
    data = []
    for plan_length in PLAN_LENGTHS:
        for question_category in QUESTION_CATEGORIES + [ALL_QUESTION_CATEGORIES_KEY]:
            for ramifications in RAMIFICATION_TYPES:
                index.append((plan_length, question_category, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
                data_columns = {}
                for domain in domains:
                    data_columns[domain] = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type, subs)
                data.append(data_columns)
    return pd.DataFrame(data, index = index)

# def df_to_latex_table(df):
#         latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
#         return latex_table[latex_table.find('\midrule'):].replace("bottomrule", 'crap').replace("\crap", '').replace("\end{tabular}", '')

# def assemble_table(results_all, answer_type, domain, score_key=None):
#     latex_table_all = ''
#     with open('latex_table_template/top') as f:
#         latex_table_all += f.read() + '\n'
#     latex_table_all += '\n'.join([df_to_latex_table(to_df(results_all, answer_type, plan_length, domain)) for plan_length in PLAN_LENGTHS])
#     with open('latex_table_template/bottom') as f:
#         latex_table_all += f.read()
#     
#     caption = f'{answer_type}, {score_key} scores for {domain}'.replace('_', ' ')
#     latex_table_all = latex_table_all.replace('REPLACE_CAPTION_KEY', caption)
#     
#     return latex_table_all


In [24]:
answer_type = TRUE_FALSE_ANSWER_TYPE #FREE_ANSWER#
score_key = F1_SCORE_KEY#'accuracy'
answer_type_ext = tf_answer_type(score_key = F1_SCORE_KEY)
stats_all = collect_stats_all(tf_answer_type(score_key = F1_SCORE_KEY))

100%|██████████| 15360/15360 [00:00<00:00, 30978.92it/s]


In [25]:
model_name = 'gemini'#'gemma-2b-it'
prompt_type = 'few_shot_1'
subs = WITHOUT_RANDOM_SUB
domains = DOMAIN_NAMES + [TRANSPORTATION_DOMAIN_KEY, NON_TRANSPORTATION_DOMAIN_KEY, ALL_DOMAINS_KEY]
# domains = [TRANSPORTATION_DOMAIN_KEY, NON_TRANSPORTATION_DOMAIN_KEY, ALL_DOMAINS_KEY]
save_key = 'all'#'transport_vs_other'
df = to_df(stats_all, model_name, answer_type, prompt_type, subs, domains)

In [26]:
df

,blocksworld,depots,driverlog,goldminer,grippers,logistics,miconic,mystery,npuzzle,satellite,spanner,visitall,zenotravel,TRANSPORTATION,NON_TRANSPORTATION,ALL_DOMAINS
"(1, object_tracking, R, T/F)",0.707143,0.797753,0.636364,0.668269,0.714286,0.741667,0.817073,0.650000,0.662420,0.688442,0.723810,0.787402,0.752874,0.729448,0.702209,0.715285
"(1, object_tracking, No R, T/F)",0.714286,0.818182,0.655844,0.673077,0.721429,0.716667,0.810976,0.650000,0.675159,0.643216,0.700000,0.787402,0.758621,0.733265,0.692603,0.712227
"(1, fluent_tracking, R, T/F)",0.670370,0.672840,0.707317,0.719444,0.809524,0.755556,0.849123,0.672222,0.696296,0.688889,0.815625,0.759259,0.722222,0.748206,0.725405,0.735738
"(1, fluent_tracking, No R, T/F)",0.659259,0.716667,0.723577,0.708333,0.814286,0.744444,0.863158,0.638889,0.703704,0.702778,0.812500,0.755556,0.718519,0.752418,0.724324,0.737136
"(1, state_tracking, R, T/F)",0.650000,0.777778,0.700000,0.750000,1.000000,0.650000,0.650000,0.500000,0.600000,0.500000,0.750000,0.500000,0.550000,0.688406,0.627119,0.660156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(19, numerical_reasoning, No R, T/F)",0.485000,0.515000,0.510000,0.495000,0.500000,0.500000,0.500000,0.500000,0.480000,0.500000,0.505000,0.505000,0.480000,0.500714,0.495000,0.498077
"(19, hallucination, R, T/F)",0.916667,0.860000,0.850000,0.878571,0.700000,0.780000,0.821429,0.830000,0.925000,0.778571,0.628571,0.683333,0.775000,0.801250,0.798718,0.800000
"(19, hallucination, No R, T/F)",0.916667,0.860000,0.800000,0.864286,0.691667,0.790000,0.850000,0.810000,0.916667,0.771429,0.628571,0.641667,0.766667,0.795000,0.787179,0.791139
"(19, ALL_CATEGORIES, R, T/F)",0.632939,0.627397,0.645267,0.645422,0.646561,0.652564,0.651460,0.616667,0.652910,0.613514,0.597368,0.640217,0.652850,0.642961,0.629263,0.636143


In [31]:
latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)

latex_table_all = r"""
\begin{table*}[h!]
\begin{adjustbox}{width=\textwidth,center}
""" + latex_table + """
\end{adjustbox}
\caption{""" + f'{model_name.replace("_", " ")}, {prompt_type.replace("_", " ")}, {subs.replace("_", " ")}, {answer_type_ext.replace("_", " ")}' + """}
\end{table*}
"""
os.makedirs(os.path.join(STATISTICS_PATH, 'tables_2'), exist_ok=True)
with open(os.path.join(STATISTICS_PATH, 'tables_2', f'{model_name}.{prompt_type}.{answer_type_ext}.{save_key}.tex'), 'w') as f:
    f.write(latex_table_all)

/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_10042/2822061382.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


In [30]:
'asdsa'.replace('a', 'b')

'bsdsb'